In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, auc
from tensorflow.keras.utils import to_categorical

In [9]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# Initialize the ResNet50 base model
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

In [4]:
# Define the number of classes dynamically based on training data or set statically
num_classes = 6 
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Data preparation
train_dir = '/content/drive/MyDrive/datasetip06/test' # please define your own path here
val_dir = '/content/drive/MyDrive/datasetip06/test' # please define your own path here

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 5400 images belonging to 6 classes.
Found 600 images belonging to 6 classes.


In [6]:
# Model training
history = model.fit(
    train_generator,
    steps_per_epoch=5399 // 32,  # Adjust based on your dataset
    epochs= 100,  # Updated as suggested
    validation_data=validation_generator,
    validation_steps=600 // 32)  # Adjust based on your dataset

Epoch 1/100
168/168 [==============================] - 672s 4s/step - loss: 1.4371 - accuracy: 0.4745 - val_loss: 2.0817 - val_accuracy: 0.1667
Epoch 2/100
168/168 [==============================] - 541s 3s/step - loss: 1.0906 - accuracy: 0.5913 - val_loss: 2.0545 - val_accuracy: 0.1632
Epoch 3/100
168/168 [==============================] - 540s 3s/step - loss: 0.9827 - accuracy: 0.6379 - val_loss: 2.0809 - val_accuracy: 0.1771
Epoch 4/100
168/168 [==============================] - 539s 3s/step - loss: 0.9000 - accuracy: 0.6606 - val_loss: 1.9553 - val_accuracy: 0.3021
Epoch 5/100
168/168 [==============================] - 539s 3s/step - loss: 0.8445 - accuracy: 0.6943 - val_loss: 1.9929 - val_accuracy: 0.4045
Epoch 6/100
168/168 [==============================] - 539s 3s/step - loss: 0.7696 - accuracy: 0.7137 - val_loss: 1.4460 - val_accuracy: 0.5260
Epoch 7/100
168/168 [==============================] - 539s 3s/step - loss: 0.7487 - accuracy: 0.7189 - val_loss: 1.4224 - val_accuracy:

In [ ]:
embedding_model = Model(inputs=base_model.input, outputs=x)  # x is the output of the last Dense layer before predictions

# Function to calculate prototypes
def calculate_prototypes(embedding_model, data_generator, num_classes):
    embeddings = []
    labels = []
    for images, label in data_generator:
        # Predict to get embeddings
        emb = embedding_model.predict(images)
        embeddings.append(emb)
        labels.append(label)

    embeddings = np.vstack(embeddings)
    labels = np.vstack(labels)

    prototypes = []
    for i in range(num_classes):  # Assuming num_classes is defined
        class_embeddings = embeddings[np.argmax(labels, axis=1) == i]
        prototype = np.mean(class_embeddings, axis=0)
        prototypes.append(prototype)

    return np.array(prototypes)

# Function for distance entropy calculation
def distance_entropy(prototypes, embeddings):
    distances = np.sqrt(((embeddings[:, np.newaxis, :] - prototypes[np.newaxis, :, :]) ** 2).sum(axis=2))
    softmax_distances = np.exp(-distances) / np.exp(-distances).sum(axis=1, keepdims=True)
    entropy = -np.sum(softmax_distances * np.log(softmax_distances + 1e-5), axis=1)
    return entropy

In [ ]:
def proto_de(features, labels):
    # Calculate the prototype (mean feature vector) for each class
    class_prototypes = {}
    for label in np.unique(labels):
        class_indices = np.where(labels == label)[0]
        class_features = features[class_indices]
        class_prototypes[label] = np.mean(class_features, axis=0)
    
    # Calculate the distance from each feature vector to its class prototype
    distances = []
    for feature, label in zip(features, labels):
        prototype = class_prototypes[label]
        distance = np.linalg.norm(feature - prototype)
        distances.append(distance)
    
    return distances

In [ ]:
def bound_de(model, features, labels):
    decision_scores = model.predict(features)  # Assuming this returns a score reflecting distance from decision boundary
    distances = np.abs(0.5 - decision_scores)  # Assuming scores are probabilities in [0, 1] for binary classification
    
    return distances


In [ ]:
def multi_branch_selection(proto_scores, bound_scores, top_k):
    # Combine Proto-DE and Bound-DE scores
    # A simple approach could be averaging the scores
    combined_scores = (np.array(proto_scores) + np.array(bound_scores)) / 2
    
    selected_indices = np.argsort(combined_scores)[-top_k:]
    
    return selected_indices

In [ ]:
def train_with_selection(model, train_data, train_labels, epochs=10, batch_size=32, top_k=100):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Step 1: Feature Extraction
        features = extract_features(model, train_data)
        
        # Step 2: Calculate Proto-DE and Bound-DE scores
        proto_scores = proto_de(features, train_labels)
        bound_scores = bound_de(model, features, train_labels)
        
        # Step 3: Select samples using Multi-Branch selection
        selected_indices = multi_branch_selection(proto_scores, bound_scores, top_k=top_k)
        selected_data = train_data[selected_indices]
        selected_labels = train_labels[selected_indices]
        
        # Step 4: Train on selected samples
        model.fit(selected_data, selected_labels, batch_size=batch_size)

In [ ]:
# Plotting training results with specified colors
plt.plot(history.history['accuracy'], color='darkblue', label='Proto-DE-Good')
plt.plot(history.history['val_accuracy'], color='orange', label='Proto-DE-Bad')
plt.plot(history.history['val_loss'], color='gray', label='Bound-DE-Good')
plt.plot(history.history['loss'], color='yellow', label='Bound-DE-Bad')
plt.plot(history.history['val_loss'], color='lightblue', label='Multi-Branch-Good')
plt.plot(history.history['loss'], color='green', label='Multi-Branch-Bad')

plt.title('Test accuracy when selecting different samples using different methods')
plt.xlabel('Test Accuracy')
plt.ylabel('Select Percentage')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), shadow=True, ncol=3)

# Adjusting legends alignment
plt.subplots_adjust(bottom=0.15)

plt.show()

In [ ]:
!pip freeze > requirements.txt
